##### Last update: april 03 2023

# Bike Share Project - Data Cleaning

In this document the data is colected, cleaned and transformed for analysis.

# Table of Contents

1. [Imports](#imports)
2. [Data Overview](#overview)
3. [Data Cleaning](#cleaning)
    * [Column re-naming](#renaming)
    * [Missing Data](#missing)
    * [Data Validation](#validation)
4. [Feature Engineering](#feature)
    * [Date](#date)
    * [Duration](#duration)
    * [Day of Week](#dow)
    * [Distance Travelled](#distance)
    * [Speed](#speed)
5. [Outliers](#outliers)
    * [Duration, speed and distance](#dynamics)
    * [Start Station Coordinates](#start_coords)
    * [End Station Coordinates](#end_coords)
6. [Verification](#verification)

# Imports <a name="imports"></a>

The usual libraries

In [3]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

# Data Overview <a name="overview"></a>

The data is stored in 12 datasets, each one concerning each of the previous 12 months from the date when this study was performed (March 2023). These contain in combination information for almost 6 million bike rides made between March 01 2022 and February 28 2023, which can be found [here](https://divvy-tripdata.s3.amazonaws.com/index.html). 

Every dataset has the same structure, with each row being a bike ride and having the following fields:

* **trip_id**: a unique identifier for each ride made with one of the bikes.
* **rideable_type**: the type of bike that was used. The types of bikes are electric, classic and docked.
* **started_at, ended_at**: the time and date the ride started/ended. 
* **start_station_name, end_station_name**: the name of the station where the trip started/ended.
* **start_station_id, end_station_id**: the code that identifies the station where the trip started/ended. 
* **start_lat, start_lng, end_lat, end_lng**: the coordinates for the station where the trip started/ended.
* **member_casual**: the type of user, casual or member.

To simplify the cleaning process, all 12 datasets are merged into a single, large dataframe. 

In [4]:
import glob

# collect all file names
csv_files = glob.glob("../data/*.csv")

# read each of them into a pd.DataFrame and put them into a list
df_list = [pd.read_csv(file) for file in csv_files]

# concatenate all dataframes, to get a single, large
# dataframe with all year's data
all_data = pd.concat(df_list, ignore_index=True)

And a copy of it is made to avoid having to load and merge all files again if something happens.

In [5]:
rides = all_data.copy()

`rides` is our complete datafrme. The following shows the structure of the data.

In [6]:
rides.sample(3)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
3808929,A97019586F0403A2,electric_bike,2022-09-22 13:35:10,2022-09-22 13:41:20,Halsted St & Polk St,TA1307000121,Throop St & Taylor St,13139,41.871795,-87.646588,41.868968,-87.659141,member
4080915,15F89969916F0372,electric_bike,2022-09-29 18:26:13,2022-09-29 18:28:31,Sheridan Rd & Buena Ave,TA1309000027,NaN,NaN,41.958493,-87.655020,41.960000,-87.650000,member
1549448,F890B691E42CCFD6,classic_bike,2022-06-25 00:27:13,2022-06-25 00:58:43,Halsted St & Roscoe St,TA1309000025,Racine Ave & Randolph St,13155,41.943670,-87.648950,41.884069,-87.656853,casual


As mentioned above, there are almost 6 million rows of data, with 13 columns containing information about the type of bike and user as well as the time and location where each trip started and ended. As can be seen below, there are some errors regarding the data types. Specifically, the dates and times are stored as string type.

In [7]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5829084 entries, 0 to 5829083
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 578.1+ MB


The only numeric fields are those for the coordinates, for which the statistical summaries are not too useful. It is worth noting, though, that there are some inconsistencies within the end station coordinates, showing latitudes and longitudes going all the way to 0°. If this was correct then there would have been bikes stationed within an area of almost 1/16th of the surface of the Earth. Who would have thought Chicago was so big?

In [8]:
rides.describe()

,start_lat,start_lng,end_lat,end_lng
count,5.829084e+06,5.829084e+06,5.823146e+06,5.823146e+06
mean,4.190217e+01,-8.764779e+01,4.190237e+01,-8.764786e+01
std,4.626503e-02,2.927663e-02,6.757085e-02,1.068591e-01
min,4.164000e+01,-8.784000e+01,0.000000e+00,-8.814000e+01
25%,4.188103e+01,-8.766150e+01,4.188103e+01,-8.766201e+01
50%,4.190000e+01,-8.764411e+01,4.190000e+01,-8.764414e+01
75%,4.193000e+01,-8.762963e+01,4.193000e+01,-8.762963e+01
max,4.207000e+01,-8.752000e+01,4.237000e+01,0.000000e+00


Finaly, the result below shows the number of duplicated rows. We can see there aren't any: each row has an unique ride id, which means it is a unique bike trip.

In [10]:
rides.duplicated(subset=["ride_id"]).sum()

0

# Data Cleaning <a name="cleaning"></a>

## Column re-naming <a name="renaming"></a>

The column names can be changed into shorter and more intuitive ones

In [ ]:
rides.rename(
    columns={
        "ride_id": "trip_id",
        "rideable_type": "bike_type",
        "started_at": "start_time",
        "ended_at": "end_time",
        "start_station_name": "start_name",
        "start_station_id": "start_id",
        "end_station_name": "end_name",
        "end_station_id": "end_id",
        "member_casual": "usertype",
    },
    inplace=True,
)

## Missing Data <a name="missing"></a>

There are values missing from some fields. The following table represents what percentage of data is missing from each field. 
There is a considerable proportion of start/end station names and id's missing and only a small amount of latitud/longitude data missing.

In [ ]:
(rides.isnull().sum() / len(rides) * 100).reset_index().rename(
    columns={"index": "Field", 0: "% of missing data"}
).set_index("Field")

The missing station names and id's can be explained with the fact that e-bikes don't need to be docked at stations, they can be locked at public bike racks, light poles, signposts, etc. 
The following confirms this is the case for most of the rides with missing names and id's

In [ ]:
for var in ["start_id", "start_name", "end_id", "end_name"]:
    print(
        f"count of missing {var}:\n",
        rides[rides[var].isnull()]
        .groupby("bike_type")["trip_id"]
        .count()
        .reset_index()
        .rename(columns={"trip_id": ""}),
        "\n",
    )

There are a few trips that were made with classic bikes that started or ended at no specified station. We can safely discard those since they account for a really small percentage of the total trips (around 0.002%). As for the trips made with e-bikes that were parked at no specific station, we can state they were parked at "no_station", with id 99999.

In [ ]:
for val in ["start_name", "end_name"]:
    rides.loc[
        (rides["bike_type"] == "electric_bike") & (rides[val].isna()), val
    ] = "No station"

for val in ["start_id", "end_id"]:
    rides.loc[
        (rides["bike_type"] == "electric_bike") & (rides[val].isna()), val
    ] = "99999"

This almost reduces the missing data to 0%

In [ ]:
print("% of missing data: ", rides.isnull().sum().sum() / len(rides) * 100)

The remainding missing values ammount to less than 0.5% of all the data, so they can be safely removed.

In [ ]:
rides_complete = rides.dropna().copy()
print("% of missing data: ", rides_complete.isnull().sum().sum() / len(rides) * 100)

## Data Validation <a name="validation"></a>

There are suposed to be around 600 stations in total, 3 types of bikes (docked, classic and e-bikes) and 2 types of users (casual and members). The following table shows that, according to the data, that is not the case for the station names and ids. Even the ratio of station names to ids is inconsistent. 

I will postpone a test for the consistency of coordinates, since as we have seen they also refer to bikes that were not parked at any station.

In [ ]:
rides_complete[
    ["bike_type", "usertype", "start_name", "start_id", "end_name", "end_id"]
].nunique()

There are a lot more stations names and id's than shoud be. A check for the consistency within stations is then needed; The following function will test if a name or id relates to multiple id's or names. It is performed only on a sample of the data since it is very timeconsuming to run it on larger scale.

In [ ]:
def not_unique(field1, field2, frac):
    """This function takes in 2 column names, field1 and field2, as well
    as the fraction, frac, of the dataframe that will be processed, and 
    returns a dictionary with the field1 values that refer to multiple
    unique values for field2 and the corresponding list of field2
    values as key-value pairs."""
    subset = rides_complete.sample(frac=frac)
    id_names = {}
    for val in ridesf[field1].unique():
        if ridesf[ridesf[field1] == val][field2].nunique() > 1:
            id_names[val] = ridesf[ridesf[field1] == val][field2].unique()
    return id_names

In [ ]:
def find_multiple_field2_values(df, field1, field2, frac):
    # Get a random subset of the dataframe to process based on the given fraction
    subset = df.sample(frac=frac)

    # Group the subset by field1 and count the number of unique field2 values in each group
    counts = subset.groupby(field1)[field2].nunique()

    # Filter the groups to only include those with multiple unique field2 values
    multiple_values = counts[counts > 1].index.tolist()

    # For each group with multiple unique field2 values, get a list of the corresponding field2 values
    result = {}
    for value in multiple_values:
        subset = df[df[field1] == value]
        field2_values = subset[field2].unique().tolist()
        result[value] = field2_values

    return result

The next two blocks of code show the number of id's that refer to two or more station names within a sample of 5% of the data, as well as the names of the stations for each of these id's.

In [ ]:
repeated_ids = find_multiple_field2_values(
    rides_complete, "start_id", "start_name", 0.05
)
len(repeated_ids)

In [ ]:
repeated_ids

Many id's do refer to different station names, but many of them have a version with "Public Rack - " or with " (Temp)" in the name and a version without. 

The next lines fix this so that the data is a bit more homogenous and the analisys more reliable.

In [ ]:
rides_complete["start_name"] = (
    rides_complete["start_name"]
    .str.replace("Public Rack - ", "")
    .str.replace(" (Temp)", "")
)
rides_complete["end_name"] = (
    rides_complete["end_name"]
    .str.replace("Public Rack - ", "")
    .str.replace(" (Temp)", "")
)

And Lets see how many station names we are left with.

In [ ]:
rides_complete[
    ["bike_type", "usertype", "start_name", "start_id", "end_name", "end_id"]
].nunique()

Its a little bit better, not a lot. The following does the same but the other way arround: shows station names that point to multiple id's.

In [ ]:
repeated_names = not_unique("start_name", "start_id", 0.05)
len(repeated_names)

In [ ]:
repeated_names

In [ ]:
del repeated_ids, repeated_names

rides_completeo pattern here. Maybe the stations changed names/id's at some point? Let's investigate how many unique station names/id's are there per month to check this hypothesis. 

For this purpose, it will be easiest to convert the dates into datetime objects and then get the year and month, but first it is necessary to verify the date format is consistent throughout the dataframe. The next lists show the dates that do not start with either the year 2022 or 2023. Since they are both empty it is safe to say that the format is consistent.

In [ ]:
[date for date in rides_complete["start_time"] if date[:4] not in ["2022", "2023"]]

In [ ]:
[date for date in rides_complete["end_time"] if date[:4] not in ["2022", "2023"]]

Next the dates are typecasted as datetime objects and the the year, month and time of day extracted since it might be interesting to study the hourly usage of the bikes in the analysis phase.

In [ ]:
rides_complete["start_time"] = pd.to_datetime(
    rides_complete["start_time"], format="%Y-%m-%d %H:%M:%S"
)
rides_complete["end_time"] = pd.to_datetime(
    rides_complete["end_time"], format="%Y-%m-%d %H:%M:%S"
)

In [ ]:
rides_complete = rides_complete.assign(year=rides_complete["start_time"].dt.year)
rides_complete = rides_complete.assign(month=rides_complete["start_time"].dt.month)
rides_complete = rides_complete.assign(time_of_day=rides_complete["start_time"].dt.hour)

And since it would be more comfortable to see the months with their names, we can make a mapping and enforce a chronological order into it.

In [ ]:
months = {
    1: "Jan",
    2: "Feb",
    3: "Mar",
    4: "Apr",
    5: "May",
    6: "Jun",
    7: "Jul",
    8: "Aug",
    9: "Sep",
    10: "Oct",
    11: "Nov",
    12: "Dec",
}
month_order = [
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
    "Jan",
    "Feb",
]
rides_complete["month"] = (
    rides_complete["month"]
    .map(months)
    .astype(pd.api.types.CategoricalDtype(categories=month_order, ordered=True))
)

With these fields it is posible to count how many station names and id's are found per month for the last 12 months.

In [ ]:
rides_complete.groupby("month")[
    ["start_name", "start_id", "end_name", "end_id"]
].nunique()

The number of stations is not accurate nor consistent on any of the fields, since the number of stations is in reallity around 600, which is half to 3/4 of the monthly reported stations on the data. To homogenize things a little bit, we proceed to make the mapping of station names to id's one to one (or as much as possible). Since there is a lower number of id's than that of names, we use first the id's to reduce the station names. Then we will use the names to further narrow down the id's. This doesn't guarantee that the relation will be exactly one to one, but it will clean up most of the mess with a single id or name value pointing to multiple names or id's.

In [ ]:
test = rides_complete.copy()

In [ ]:
# for ids in test["start_id"].unique():
#     if ids != "99999":
#         if len(test[test["start_id"] == ids]["start_name"].unique()) > 1:
#             test.loc[test["start_id"] ==
#                                ids, "start_name"] = np.sort(test.loc[test["start_id"]
#                                                                                == ids, "start_name"].unique())[0]
mask = test["start_id"] != "99999"
has_multiple_names = test.groupby("start_id")["start_name"].transform("nunique") > 1
mask &= has_multiple_names
unique_names = test[mask].groupby("start_id")["start_name"].apply(lambda x: x.unique())
first_name = unique_names.apply(lambda x: np.sort(x)[0])
test.loc[mask, "start_name"] = first_name

In [ ]:
test.groupby("month")[["start_name", "start_id", "end_name", "end_id"]].nunique()

In [ ]:
test.groupby("month")[["start_name", "start_id", "end_name", "end_id"]].nunique()

In [ ]:
unique_names

In [ ]:
# for name in test["start_name"].unique():
#     if name != "No station":
#         if len(test[test["start_name"] == name]["start_id"].unique()) > 1:
#             test.loc[test["start_name"] ==
#                                name, "start_id"] = np.sort(test.loc[test["start_name"]
#                                                                               == name, "start_id"].unique())[0]
mask = test["start_name"] != "No station"
has_multiple_ids = test.groupby("start_name")["start_id"].transform("nunique") > 1
mask &= has_multiple_ids
unique_ids = test[mask].groupby("start_name")["start_id"].apply(lambda x: x.unique())
first_id = unique_ids.apply(lambda x: np.sort(x)[0])
test.loc[mask, "start_id"] = first_id

In [ ]:
test.groupby(["month"])["start_name", "start_id", "end_name", "end_id"].nunique()

And now for the end stations

In [ ]:
names = [*test["start_name"].unique()]
id_list = [*test["start_id"].unique()]

In [ ]:
name_to_id = dict(test.groupby("start_name")["start_id"].unique())
name_to_id

In [ ]:
test.loc[:, "end_id"] = test["end_name"].map(name_id)

# for ids in test["end_id"].unique():
#     if ids in id_list:
#         test.loc[test["end_id"] ==
#                            ids, "end_name"] = test[test["start_id"]
#                                                              ==  ids]["start_name"].unique()[0]
# create a mask for the end ids to update

In [ ]:
for name in rides_complete["end_name"].unique():
    if name in names:
        rides_complete.loc[
            rides_complete["end_name"] == name, "end_id"
        ] = rides_complete[rides_complete["start_name"] == name]["start_id"].unique()[0]

In [ ]:
del names, id_list

After this we at least have some consistency: the number of names and id's for start/end stations is the same within each month.

In [ ]:
rides_complete.groupby(["month"])[
    "start_name", "start_id", "end_name", "end_id"
].nunique()

And the total number of names and ids for start and end stations are almost exactly the same.

In [ ]:
rides_complete[["start_name", "end_name", "start_id", "end_id"]].nunique()

In [ ]:
rides_homogenized = rides_complete.copy()

There still are a lot more stations than there should be but at least now the relation of station names to id's is (almost) 1 to 1 and they are standardized for start and end stations.


Next, there are multiple sets of coordinates for each station, differing only by a small amount between them, as can be seen below for one single station having over 5000 values for its latitude. 

In [ ]:
rides_homogenized[["start_lat", "start_lng", "end_lat", "end_lng"]].nunique()

In [ ]:
rides_homogenized[rides_homogenized["start_name"] == "State St & Kinzie St"][
    "start_lat"
].nunique()

In [ ]:
rides_homogenized[rides_homogenized["start_id"] == "13042"]["start_lat"].head(5)

Before we implement any solution it might be a good idea to address the issue with the end station coordinate ranges.

In [ ]:
rides_homogenized[["start_lat", "start_lng", "end_lat", "end_lng"]].agg(
    ["max", "min"]
).transpose()

The ranges shoud be roughly the same but for some reason it is very different for the end stations. The coordinate range shown above accounts for roughly 1/16th of the Earth's surface area, so there is something fishy going on. 

After checking the data with trips ending in coordinates outside the range for the start stations we find that there is a minimal amount of rides that fall outside that range, and their coordinates all have a value of 0.0, 0.0. Also, all of them ended at Green St & Madison Ave* station.

In [ ]:
rides_homogenized[
    (rides_homogenized["end_lat"] < 40) | (rides_homogenized["end_lng"] > -87)
]

It is easy to confirm that this is not the case for all records of that station; there are other instances where the coordinates do fall within the range, so we can use any of those as a replacement.

In [ ]:
rides_homogenized[rides_homogenized["end_name"] == "Green St & Madison Ave*"].head(2)

In [ ]:
rides_homogenized.loc[rides_homogenized["end_lat"] < 40, "end_lat"] = 41.881827
rides_homogenized.loc[rides_homogenized["end_lng"] > -86.5, "end_lng"] = -87.648832

With this we are left with a more sensible range of coordinates for all stations.

In [ ]:
rides_homogenized[["start_lat", "start_lng", "end_lat", "end_lng"]].agg(
    ["min", "max"]
).transpose()

Now that this has been taken care of, one posible solution to the issue of the multiple sets of coordinates per station is to use the average of each stations coordinates to figure out the average coordinate "distance" between them. This will allow us to determine at which decimal point could we round up the coordinates without mixing up different stations. 

In [ ]:
rides_coords = (
    rides_homogenized[rides_homogenized["start_name"] != "No station"]
    .groupby("start_name")[["start_lat", "start_lng"]]
    .mean()
    .rename(columns={"start_lat": "lat", "start_lng": "lng"})
)

In [ ]:
rides_coords.sort_values("lat")["lat"].diff().mean()

In [ ]:
rides_coords.sort_values("lng")["lng"].diff().mean()

In [ ]:
del rides_coords

From these results it looks like the 5th decimal point would give a good amount of precision to differentiate stations, while elimination many of the small vatiations within each station. 

In [ ]:
for coord in ["start_lat", "start_lng"]:
    rides_homogenized[coord] = round(rides_homogenized[coord], 5)

for coord in ["end_lat", "end_lng"]:
    rides_homogenized[coord] = round(rides_homogenized[coord], 5)

And here it is again the number of unique values for each field after these modifications.

In [ ]:
rides_homogenized[
    [
        "start_name",
        "end_name",
        "start_id",
        "end_id",
        "start_lat",
        "end_lat",
        "start_lng",
        "end_lng",
    ]
].nunique()

It is by no means perfect, but it will be good enough for analysis. Some further study on the actual stations, their current names and ids as well as their locations, could help make the data more congruent with real life. 

In [ ]:
rides_clean = rides_homogenized.copy()

# Feature Engineering <a name="feature"></a>

There are a couple fields that might be useful to have, such as the pure date for each trip, the day of week when it happened and the distance travelled. The next subsections are dedicated to these fields. 

## Date <a name="date"></a>

We already have a datetime object from which we can extract the date.

In [ ]:
rides_clean["date"] = rides_clean["start_time"].apply(lambda x: x.date())

## Duration <a name="duration"></a>

In short, how much time passed from the moment the ride started to the moment it ended.

In [ ]:
rides_clean["duration"] = rides_clean["end_time"] - rides_clean["start_time"]

For consistency we check for any negative durations, since they don't really make sense in this scenario, as well as really long ones. The following lines show that there was a 23 day ride and, more importantly, there is in fact at least one trip with a negative duration.

In [ ]:
rides_clean["duration"].max()

In [ ]:
rides_clean["duration"].min()

Since there is at least one acount of a trip with a negative duration, it is worth studying if this is an extended issue or if it is just a few exceptions. The percentage of trips that have a negative duration is

In [ ]:
len(rides_clean[rides_clean["duration"] < timedelta(seconds=0)]) / len(
    rides_clean
) * 100

Since the percentage is minimal, we can safely discard these records. Also, the bike sharing website sugests that the data should be purged of rows with a duration of less than 60 seconds, since those could be fake starts or servicing tasks made by the staff. The next line of code removes both cases at once.

In [ ]:
rides_clean = rides_clean[rides_clean["duration"] >= timedelta(seconds=60)]

And for visualization purposes the duration as an int field might make things easier, so the next line calculates the duration of the ride in minutes.

In [ ]:
rides_clean["minutes"] = rides_clean["duration"].apply(
    lambda x: round(x.total_seconds() / 60, 1)
)

## Day of Week <a name="dow"></a>

Different types of users might use the bikes on different days of the week. To test this we will need to know the day of the week for each trip. The date for the beginning of the trip is used.

In [ ]:
days = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}
days_order = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]

In [ ]:
rides_clean["day_of_week"] = (
    rides_clean["start_time"]
    .map(lambda x: x.weekday())
    .map(days)
    .astype(pd.api.types.CategoricalDtype(categories=days_order, ordered=True))
)

## Distance Travelled <a name="distance"></a>

The distance from start to end station can be found using pythagoras theorem,  but to convert from latitudes and longitudes to meters or km it requires an additional step. 

### Latitude
The north-south location is determined by its latitude, which is meassured from the equator, from 0° to $\pm$90°. 
The vertical distance, $d_{lat}$, approximating the Earth as a perfect sphere, is given by Earth's radius, $R\approx 6371 km$, and by the difference in latitude, $\Delta\phi$, measured in radians, that is

<p style="text-align: center;"> $d_{lat} = R\Delta\phi \frac{\pi}{180}$ </p>

It comes down to arround 111.195 km per degree of latitude.

### Longitude
For longitude - that is, the east-west location - it is a little bit trickier: since the distance between each longitude line decreases as you approach the poles, there is an additional factor. An easy way to understand it is that it is basically the same as for latitude, we are after all meassuring an arc length, but the radius of the circle is smaller. It is smaller, in fact, by a factor of $cos(\phi)$. So, if $\theta$ is is the longitude, then the distance between two given longitudes is

<p style="text-align: center;"> $d_{lng} = R\Delta\theta cos(\phi) \frac{\pi}{180}$ </p>

Which, again, can be simplified to 111.195$cos(\phi)$ km per degree of longitude.

The distance $d$ travelled, then, is approximately

<p style="text-align: center;"> $d\approx \sqrt{d_{lat}^2+d_{lng}^2}$ </p>

and it is valid for short distances, where we can neglect the curvature of the earth. Distances within a city certainly meet this criteria.

So lets define some functions to make this a bit cleaner.

In [ ]:
def lat_distance(df):
    return (111.195 * (df["end_lat"] - df["start_lat"])) ** 2


def lng_distance(df):
    cos = np.cos(df["start_lat"])
    return (111.195 * cos * (df["end_lng"] - df["start_lng"])) ** 2


def coord_distance(df):
    distance_squared = lat_distance(df) + lng_distance(df)
    return np.sqrt(distance_squared)

also, we only need the precission to go as far as meters, so the distance is rounded to 2 decimal places.

In [ ]:
rides_clean["distance"] = round(coord_distance(rides_clean), 2)
rides_clean.sample(2)

Looks good. For a quick verification we get the range of distances traveled. It goes from 0 km, for trips that ended right where they started, to 36.03 km, which is a reasonable distance to ride in a few hours.

In [ ]:
rides_clean["distance"].agg(["min", "max"])

## Speed <a name="speed"></a>

It might be interesting to check if there are differences between the average speeds of members vs casual users. The average speed of each trip is given by the total distance traveled divided by the duration of the trip. Since the data has been purged from trips with a duration of less than a minute, there is no danger of dividing by zero.

In [ ]:
rides_clean.loc[rides_clean["minutes"] > 0, "speed"] = round(
    60 * rides_clean["distance"] / rides_clean["minutes"], 2
)

In [ ]:
rides_with_features = rides_clean.copy()

# Outliers <a name="outliers"></a>

There is a 23 day trip in the data. This is enough evidence to sugest there may be some outliers lying arround. 

### duration, speed and distance <a name="dynamics"></a>

The next tables show some descrptive summary statistics for duration in minutes, distance traveled and speed.

We see that the mean duration is considerably far appart from its median, sugesting there data for this field is highly skewed to the right. This is further verified by the really high standard deviation. Speed is better, but still has a somewhat large standard deviation. The plots below clarify a bit what is the situation with the speeds. For the distance, again, the difference between the mean and the median is considerable, and the standard deviation somewhat large, but nothing huge. 

The main concern here is then the duration, although all three fields need a closer look.

In [ ]:
rides_with_features[["minutes", "speed", "distance"]].agg(["mean", "median", "std"])

The next plots show, for the same three variables, its interquartile ranges, with the boxplots, as well as the overall distribution of their values, shown in the histograms. 

It is immediately noticeable that all 3 variables have numerous and significant outliers, the most dramatic being the duration. A close up for each plot shows the main body of the data.

In [ ]:
fig, ax = plt.subplots(3, 2)

fig.set_figwidth(12)
fig.set_figheight(10)

sns.boxplot(ax=ax[0, 0], data=rides_with_features["minutes"])
ax2 = plt.axes([0.17, 0.71, 0.1, 0.12])
sns.boxplot(
    ax=ax2, data=rides_with_features[rides_with_features["minutes"] < 120]["minutes"]
)
ax2.set_ylim(0, 120)
ax[0, 0].set_ylabel("Duration", size=15)

sns.histplot(ax=ax[0, 1], data=rides_with_features["minutes"], bins=50)
ax2 = plt.axes([0.69, 0.72, 0.15, 0.12])
sns.histplot(
    ax=ax2,
    data=rides_with_features[rides_with_features["minutes"] < 120]["minutes"],
    bins=50,
)
ax2.set_xlim(0, 120)

sns.boxplot(ax=ax[1, 0], data=rides_with_features["speed"])
ax2 = plt.axes([0.17, 0.46, 0.1, 0.12])
sns.boxplot(
    ax=ax2, data=rides_with_features[rides_with_features["speed"] < 40]["speed"]
)
ax2.set_ylim(0, 40)
ax[1, 0].set_ylabel("Speed", size=15)

sns.histplot(ax=ax[1, 1], data=rides_with_features["speed"], bins=50)
ax2 = plt.axes([0.69, 0.46, 0.15, 0.12])
sns.histplot(
    ax=ax2,
    data=rides_with_features[rides_with_features["speed"] < 40]["speed"],
    bins=50,
)
ax2.set_xlim(0, 40)

sns.boxplot(ax=ax[2, 0], data=rides_with_features["distance"])
ax2 = plt.axes([0.17, 0.18, 0.1, 0.12])
sns.boxplot(
    ax=ax2, data=rides_with_features[rides_with_features["distance"] < 40]["distance"]
)
ax2.set_ylim(0, 10)
ax[2, 0].set_ylabel("Distance", size=15)

sns.histplot(ax=ax[2, 1], data=rides_with_features["distance"], bins=50)
ax2 = plt.axes([0.68, 0.18, 0.15, 0.12])
sns.histplot(
    ax=ax2,
    data=rides_with_features[rides_with_features["distance"] < 40]["distance"],
    bins=50,
)
ax2.set_xlim(0, 15)

Since we are interested in the majority of users, we can trim the data to avoid outliers. The code below shows that by keeping the records that lie within 3 standard deviations of the mean for duration and speed and trips with a length of less than 10 km we are throwing out at most 1.638% of the data. 

In [ ]:
mins = len(rides_with_features[rides_with_features["minutes"] > 120])
speed = len(rides_with_features[rides_with_features["speed"] > 24])
dist = len(rides_with_features[rides_with_features["distance"] > 10])

100 * (mins + speed + dist) / len(rides_with_features)

So lets proceed with filtering out those records (I didn't use exacly 3 standard deviations since part of the tails of the histograms were high enough to stil be statistically significant. For the duration variable it was the opposite, so I used less than 3 std)

In [ ]:
rides_filtered = rides_with_features[rides_with_features["minutes"] <= 100]
rides_filtered = rides_filtered[rides_filtered["speed"] <= 30]
rides_filtered = rides_filtered[rides_filtered["distance"] <= 10]

In [ ]:
fig, ax = plt.subplots(3, 2)

fig.set_figwidth(12)
fig.set_figheight(10)

sns.boxplot(ax=ax[0, 0], data=rides_filtered, x="minutes")
ax[0, 0].set_ylabel("Duration", size=15)

sns.histplot(ax=ax[0, 1], data=rides_filtered["minutes"], bins=50)

sns.boxplot(ax=ax[1, 0], data=rides_filtered, x="speed")
ax[1, 0].set_ylabel("Speed", size=15)

sns.histplot(ax=ax[1, 1], data=rides_filtered["speed"], bins=50)

sns.boxplot(ax=ax[2, 0], data=rides_filtered, x="distance")
ax[2, 0].set_ylabel("Distance", size=15)

sns.histplot(ax=ax[2, 1], data=rides_filtered["distance"], bins=50)

### Start station coordinates <a name="start_coords"></a>

In [ ]:
rides_filtered[["start_lat", "start_lng"]].agg(["mean", "median", "std", "min", "max"])

In [ ]:
fig, ax = plt.subplots(2, 2)

fig.set_figwidth(10)
fig.set_figheight(10)

sns.boxplot(ax=ax[0, 0], data=rides_filtered["start_lat"])
sns.histplot(ax=ax[0, 1], data=rides_filtered, y="start_lat", bins=50)
ax[0, 1].set_ylim(41.5, 42.3)
ax[0, 0].set_ylabel("Start Latitude", size=14)

sns.boxplot(ax=ax[1, 0], data=rides_filtered, x="start_lng")
sns.histplot(ax=ax[1, 1], data=rides_filtered["start_lng"], bins=50)
ax[1, 1].set_xlim(-87.9, -87.5)
ax[1, 0].set_ylabel("Start Longitude", size=14)

### End station coordinates <a name="end_coords"></a>

In [ ]:
rides_filtered[["end_lat", "end_lng"]].agg(["mean", "median", "std", "min", "max"])

In [ ]:
fig, ax = plt.subplots(2, 2)

fig.set_figwidth(10)
fig.set_figheight(10)

sns.boxplot(ax=ax[0, 0], data=rides_filtered["end_lat"])
sns.histplot(ax=ax[0, 1], data=rides_filtered, y="end_lat", bins=50)
ax[0, 1].set_ylim(41.5, 42.3)
ax[0, 0].set_ylabel("End Latitude", size=14)

sns.boxplot(ax=ax[1, 0], data=rides_filtered, x="end_lng")
sns.histplot(ax=ax[1, 1], data=rides_filtered["end_lng"], bins=50)
ax[1, 1].set_xlim(-87.9, -87.5)
ax[1, 0].set_ylabel("End Longitude", size=14)

The range for the end station coordinates is larger than that of the start station's, and there are more evident outliers, especially for the longitude. To keep things simple we can restrict the range of coordinates to be that of the start stations.

In [ ]:
min_lat = rides_filtered["start_lat"].min()
max_lat = rides_filtered["start_lat"].max()
min_lng = rides_filtered["start_lng"].min()
max_lng = rides_filtered["start_lng"].max()

rides_filtered = rides_filtered[
    (rides_filtered["end_lat"] >= min_lat) & (rides_filtered["end_lat"] <= max_lat)
]
rides_filtered = rides_filtered[
    (rides_filtered["end_lng"] >= min_lng) & (rides_filtered["end_lng"] <= max_lng)
]

In [ ]:
rides_filtered[["end_lat", "end_lng"]].agg(["mean", "median", "std", "min", "max"])

# Verification <a name="verification"></a>

Now that we have a final cleaned version of the data it is a good time to verify that there are no missing values and that the number of unique values for each field and the ranges for the numeric variables make sense.

In [ ]:
rides_filtered.isnull().sum().sum()

In [ ]:
rides_filtered.nunique()

In [ ]:
rides_filtered.describe()

Now everything is filtered and a bit more consistent, although there are some instances of people biking at more than 200 km/h. This is definitely something to look at later. 

For now, to clear up things a bit we can delete all versions of the dataframe that wont be used anymore, and reinstate the original name.

In [ ]:
rides = rides_filtered.copy()
del rides_filtered, rides_homogenized, rides_with_features, rides_clean, rides_complete

And, finally, we need to save the data for analysis.

In [ ]:
rides.to_csv("../data/cleaned/rides.csv", index=False)

In total we lost %3.900 of the data during cleaning. Not too bad. 

In [ ]:
(all_data.shape[0] - rides.shape[0]) / all_data.shape[0] * 100